In [1]:
from google.colab import drive
import os

# Подключаем Google Drive
drive.mount('/content/drive')

GDRIVE_PATH = "/content/drive/MyDrive/colab_pdf_model"

# Создаём папку, если её нет
os.makedirs(GDRIVE_PATH, exist_ok=True)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 74.6 MB/s eta 0:00:00


In [44]:
!pip install pymupdf

In [4]:
import os
import pickle
import sqlite3
import pdfplumber  # ✅ Заменили PyPDF2 на pdfplumber
import numpy as np
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [5]:
# 🗂 Пути для хранения файлов
GDRIVE_PATH = "/content/drive/MyDrive/colab_pdf_model"
PDF_FOLDER = "/content/drive/MyDrive/pdf_micro2"
TOKENIZER_PATH = os.path.join(GDRIVE_PATH, "tokenizer.pkl")
MODEL_PATH = '/content/drive/MyDrive/colab_pdf_model/model.keras'  # Убедитесь, что файл действительно находится здесь
model = load_model(MODEL_PATH)
DB_PATH = os.path.join(GDRIVE_PATH, "dataset.db")

In [6]:
# ✅ Создаём папки, если их нет
os.makedirs(GDRIVE_PATH, exist_ok=True)
os.makedirs(PDF_FOLDER, exist_ok=True)

In [7]:
# 🔄 Функция загрузки или создания токенизатора
def load_or_create_tokenizer():
    if os.path.exists(TOKENIZER_PATH):
        print("📥 Загружаем существующий токенизатор...")
        with open(TOKENIZER_PATH, "rb") as f:
            tokenizer = pickle.load(f)
    else:
        print("🆕 Создаём новый токенизатор...")
        tokenizer = Tokenizer()
    return tokenizer

In [8]:
# 🔄 Функция сохранения токенизатора
def save_tokenizer(tokenizer):
    with open(TOKENIZER_PATH, "wb") as f:
        pickle.dump(tokenizer, f)
    print("💾 Токенизатор сохранён.")

In [9]:
# 🔄 Функция загрузки или создания модели
def load_or_create_model(vocab_size):
    if os.path.exists(MODEL_PATH):
        print("📥 Загружаем обученную модель...")
        model = load_model(MODEL_PATH)
    else:
        print("🆕 Создаём новую модель...")
        model = Sequential([
            Embedding(vocab_size, 128, input_length=100),
            LSTM(128, return_sequences=True),
            LSTM(64),
            Dense(64, activation="relu"),
            Dense(vocab_size, activation="softmax")
        ])
        model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model


In [10]:
# 📄 Функция чтения текста из PDF с pdfplumber
def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                text += page.extract_text() + "\n"
    except Exception as e:
        print(f"⚠ Ошибка при обработке {pdf_path}: {e}")
    return text.strip()

In [11]:
# 📂 Собираем текст из PDF
all_texts = []
pdf_files = [f for f in os.listdir(PDF_FOLDER) if f.endswith(".pdf")]
if not pdf_files:
    print("❌ В папке нет PDF-файлов. Загрузите файлы в Google Drive → pdf_micro.")

for filename in pdf_files:
    pdf_path = os.path.join(PDF_FOLDER, filename)
    text = extract_text_from_pdf(pdf_path)
    if text:
        all_texts.append(text)


In [12]:
# ✅ Загружаем или создаём токенизатор
tokenizer = load_or_create_tokenizer()

📥 Загружаем существующий токенизатор...


In [13]:
 # Обновляем токенизатор
if all_texts:
    tokenizer.fit_on_texts(all_texts)
    save_tokenizer(tokenizer)

💾 Токенизатор сохранён.


In [14]:
# 📌 Преобразуем текст в последовательности
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(all_texts)
padded_sequences = pad_sequences(sequences, maxlen=100, padding="post")

In [15]:
# 🎯 Загружаем или создаём модель
vocab_size = len(word_index) + 1
model = load_or_create_model(vocab_size)

📥 Загружаем обученную модель...


In [16]:
# 💾 Функция сохранения модели
def save_model():
    model.save(MODEL_PATH)
    print("💾 Модель сохранена в Google Drive.")

In [17]:
# 🛠 Сохраняем текст в базу данных SQLite
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()
cursor.execute("CREATE TABLE IF NOT EXISTS documents (id INTEGER PRIMARY KEY, text TEXT)")
for text in all_texts:
    cursor.execute("INSERT INTO documents (text) VALUES (?)", (text,))
conn.commit()
conn.close()
print("📂 Данные сохранены в dataset.db")

📂 Данные сохранены в dataset.db


In [76]:
# Проверим размерность обучающих данных
print("Размерность обучающих данных:", X_train.shape)  # Должно быть (n_samples, 10)

# Проверим размерность данных, которые нужно преобразовать
print("Размерность текущих данных:", X.shape)  # Должно быть (n_samples, 10)

# Проверим размерность данных X
print("Размерность данных X:", X.shape)

# Проверим размерность данных, на которых обучался скейлер
print("Размерность данных, на которых обучался скейлер:", scaler.n_features_in_)

# Если размерности не совпадают, можно привести их к одинаковому виду:
# Например, если в X отсутствуют некоторые признаки, можно добавить их с нулями или соответствующими значениями.
if X.shape[1] != scaler.n_features_in_:
    print("Размерности данных и скейлера не совпадают. Приводим данные к нужному виду.")
    # Возможно, добавим недостающие признаки с нулями
    missing_columns = scaler.n_features_in_ - X.shape[1]
    X = np.hstack([X, np.zeros((X.shape[0], missing_columns))])  # Добавляем нули, если признаки отсутствуют

# Печатаем все колонки в датасете для проверки
print("Колонки в датасете:", data.columns.tolist())

# Применение токенизатора: Проверим, что tokenizer обучен
if hasattr(tokenizer, 'word_index'):
    print("Словарь индексов токенизатора:", tokenizer.word_index)
else:
    print("Ошибка: Токенизатор не обучен")

# Пример текста для предсказания
seed_text = "java"

# Преобразуем seed_text в последовательность индексов
sequence = tokenizer.texts_to_sequences([seed_text])
print("Индексированная последовательность:", sequence)  # Выводим последовательность после преобразования

Размерность обучающих данных: (8, 8)
Размерность текущих данных: (10, 8)
Размерность данных X: (10, 8)
Размерность данных, на которых обучался скейлер: 8
Колонки в датасете: ['id', 'text']
Словарь индексов токенизатора: {'<OOV>': 1, 'the': 2, 'quick': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'hello': 10, 'world': 11}
Индексированная последовательность: [[1]]


In [77]:
import os
import numpy as np
import pandas as pd
import joblib
import sqlite3
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.models import load_model

# 🗂 Пути
GDRIVE_PATH = "/content/drive/MyDrive/colab_pdf_model"
MODEL_PATH = os.path.join(GDRIVE_PATH, "model.keras")
SCALER_PATH = os.path.join(GDRIVE_PATH, "new_scaler.pkl")
DB_PATH = os.path.join(GDRIVE_PATH, "dataset.db")

# Создаём директорию, если её нет
os.makedirs(GDRIVE_PATH, exist_ok=True)

# 🔹 Загружаем данные из базы SQLite
if not os.path.exists(DB_PATH):
    raise FileNotFoundError(f"❌ Файл базы данных {DB_PATH} не найден! Проверь путь.")

conn = sqlite3.connect(DB_PATH)

# Проверяем список таблиц
query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql(query, conn)

print("📌 Таблицы в базе данных:")
print(tables)

# Выбираем данные из таблицы documents (поменяй на нужное имя)
TABLE_NAME = "documents"  # Укажи правильное имя таблицы
if TABLE_NAME not in tables["name"].values:
    raise ValueError(f"❌ Таблица '{TABLE_NAME}' не найдена в базе данных!")

# Загружаем данные
data = pd.read_sql(f"SELECT * FROM {TABLE_NAME};", conn)

# Закрываем соединение
conn.close()

print(f"✅ Данные загружены из таблицы {TABLE_NAME}")
print(data.head())  # Выводим первые строки

# 🔹 Разделяем признаки и целевую переменную
target_column = "id"  # ❗ Замени на правильное имя целевой переменной
if target_column not in data.columns:
    raise ValueError(f"❌ В датасете нет колонки {target_column}! Проверь названия столбцов.")

X = data.drop(columns=[target_column])
y = data[target_column]

# 🔹 Преобразуем текстовые данные в числовые (если есть такие)
text_columns = X.select_dtypes(include=['object']).columns
if len(text_columns) > 0:
    print(f"📌 Преобразуем текстовые столбцы: {text_columns}")
    # Используем OneHotEncoder для преобразования текстовых данных в числовые
    transformer = ColumnTransformer([
        ('text', OneHotEncoder(), text_columns),
    ], remainder='passthrough')  # Оставляем остальные столбцы без изменений

    X = transformer.fit_transform(X)
    # Сохраняем трансформатор для дальнейшего использования
    joblib.dump(transformer, SCALER_PATH.replace("scaler", "transformer"))
    print(f"✅ Трансформатор сохранён в {SCALER_PATH.replace('scaler', 'transformer')}")
else:
    print("📌 Текстовых данных нет, пропускаем преобразование.")

# 🔹 Нормализация данных
if os.path.exists(SCALER_PATH):
    scaler = joblib.load(SCALER_PATH)
    print("📌 Загружен существующий scaler")
else:
    scaler = StandardScaler()
    scaler.fit(X)  # Применяем масштабирование ко всему набору данных до разделения
    joblib.dump(scaler, SCALER_PATH)
    print(f"✅ Scaler сохранен в {SCALER_PATH}")

X = scaler.transform(X)  # Применяем нормализацию ко всем данным

# 🔹 Разделение на обучающую и валидационную выборки
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"✅ Размеры выборок: X_train: {X_train.shape}, X_val: {X_val.shape}")

# Преобразуем данные с использованием трансформатора
if os.path.exists(SCALER_PATH.replace("scaler", "transformer")):
    transformer = joblib.load(SCALER_PATH.replace("scaler", "transformer"))
    print("📌 Загружен существующий трансформатор")
else:
    raise ValueError("❌ Не удалось найти трансформатор! Проверь путь.")

# Применяем трансформатор к данным
X_train_transformed = transformer.transform(X_train)
X_val_transformed = transformer.transform(X_val)

# Преобразуем разреженные матрицы в плотные
X_train_transformed = X_train_transformed.toarray()  # Плотная матрица
X_val_transformed = X_val_transformed.toarray()  # Плотная матрица

# Применяем нормализацию только после трансформации
X_train_transformed = scaler.transform(X_train_transformed)
X_val_transformed = scaler.transform(X_val_transformed)


# 🔹 Определяем архитектуру модели
def build_model(input_shape):
    model = keras.Sequential([
        Dense(128, activation=tf.keras.activations.swish, input_shape=input_shape),
        BatchNormalization(),
        Dropout(0.3),
        Dense(64, activation=tf.keras.activations.swish),
        BatchNormalization(),
        Dropout(0.3),
        Dense(32, activation=tf.keras.activations.swish),
        Dense(1, activation="linear")  # Для регрессии
    ])

    model.compile(optimizer=Adam(learning_rate=1e-3),
                  loss=tf.keras.losses.Huber(delta=1.5),
                  metrics=["mae"])
    return model

# 🔹 Загружаем модель (если уже обучена)
if os.path.exists(MODEL_PATH):
    model = load_model(MODEL_PATH)
    print(f"📌 Загружена существующая модель из {MODEL_PATH}")
else:
    input_shape = (X_train.shape[1],)
    model = build_model(input_shape)
    print("✅ Создана новая модель")

# 🔹 Коллбэки
callbacks = [
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, min_lr=1e-6, verbose=0),
    ModelCheckpoint(MODEL_PATH, save_best_only=True, monitor="val_loss", verbose=1)
]

# 🔹 Запускаем обучение
history = model.fit(X_train_transformed, y_train,
                    validation_data=(X_val_transformed, y_val),
                    epochs=70,
                    batch_size=64,
                    callbacks=callbacks)

# 🔹 Сохраняем модель
model.save(MODEL_PATH)
print(f"✅ Модель сохранена в {MODEL_PATH}")

# 🔹 Построение графиков обучения
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_mae = history.history['mae']
val_mae = history.history['val_mae']

plt.figure(figsize=(12, 6))
plt.plot(train_loss, label="Train Loss", color="blue")
plt.plot(val_loss, label="Validation Loss", color="orange")
plt.plot(train_mae, label="Train MAE", color="green")
plt.plot(val_mae, label="Validation MAE", color="red")
plt.xlabel("Epochs")
plt.ylabel("Loss / MAE")
plt.title("Model Training History")
plt.legend()
plt.grid(True)
plt.show()


📌 Таблицы в базе данных:
        name
0  documents
✅ Данные загружены из таблицы documents
   id                                               text
0   1  Алексей Васильев\nJAVA\nД ЛЯ ВСЕХ\nКНИГА О ПОП...
1   2  Алексей Васильев\nJAVA\nД ЛЯ ВСЕХ\nКНИГА О ПОП...
2   3  Алексей Васильев\nJAVA\nД ЛЯ ВСЕХ\nКНИГА О ПОП...
3   4  (cid:1)(cid:3)(cid:4)(cid:5)(cid:6)(cid:7)(cid...
4   5  Дэвид Копец\nКлассические задачи\nComputer Sci...
📌 Преобразуем текстовые столбцы: Index(['text'], dtype='object')
✅ Трансформатор сохранён в /content/drive/MyDrive/colab_pdf_model/new_transformer.pkl
📌 Загружен существующий scaler
✅ Размеры выборок: X_train: (8, 8), X_val: (2, 8)
📌 Загружен существующий трансформатор


ValueError: X has 8 features, but ColumnTransformer is expecting 1 features as input.

In [39]:
import os
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Пути для загрузки модели
GDRIVE_PATH = "/content/drive/MyDrive/colab_pdf_model"
MODEL_PATH = os.path.join(GDRIVE_PATH, "model.keras")  # Указываем правильный путь к модели .keras
TOKENIZER_PATH = os.path.join(GDRIVE_PATH, "tokenizer.pkl")

# Загружаем токенизатор
with open(TOKENIZER_PATH, "rb") as f:
    tokenizer = pickle.load(f)

# Загружаем модель
model = load_model(MODEL_PATH)
print("Модель и токенизатор загружены!")

# Функция для сохранения модели
def save_model(model, path='model.keras'):
    """Сохраняет модель по заданному пути."""
    model.save(path)
    print(f"Модель сохранена в {path}")

# Основная функция для генерации текста
def generate_text(seed_text, next_words=50, save_after_generation=True):
    """Генерирует текст на основе начальной строки (seed_text) и сохраняет модель после генерации."""
    max_sequence_length = 10  # Замените на нужную длину, если она отличается

    for _ in range(next_words):
        # Преобразуем текст в последовательность
        sequence = tokenizer.texts_to_sequences([seed_text])

        # Паддируем последовательность до нужной длины
        padded_sequence = pad_sequences(sequence, maxlen=max_sequence_length, padding="pre")

        # Предсказываем следующее слово
        predicted_index = np.argmax(model.predict(padded_sequence), axis=-1)[0]

        # Находим слово по индексу
        word = None
        for w, index in tokenizer.word_index.items():
            if index == predicted_index:
                word = w
                break

        # Добавляем слово к предложению
        if word:
            seed_text += " " + word
        else:
            break  # Если слово не найдено, останавливаемся

    if save_after_generation:
        # Сохраняем модель после генерации текста
        save_model(model)

    return seed_text

# Пример использования
generated_text = generate_text("println")
print("Сгенерированный текст:", generated_text)

# Сохранение модели в формате .keras (если это нужно по завершению работы)
save_model(model, 'model.keras')

# Если нужно скачать модель из Google Colab
#from google.colab import files
#files.download('model.keras')  # Скачиваем модель на ваш локальный компьютер


Модель и токенизатор загружены!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
Модель сохранена в model.keras
Сгенерированный текст: println
Модель сохранена в model.keras


In [134]:
# Скачиваем модель
files.download(MODEL_PATH)

# Скачиваем токенизатор
files.download(TOKENIZER_PATH)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [74]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Пример текста
seed_text = "The quick brown fox jumps"

# Инициализация токенизатора с учётом OOV токена
tokenizer = Tokenizer(oov_token='<OOV>')
# Тренируем токенизатор на ваших данных
text_data = ["The quick brown fox jumps", "Over the lazy dog", "Hello world"]  # Пример данных для обучения
tokenizer.fit_on_texts(text_data)

# Преобразуем текст в последовательность индексов
sequence = tokenizer.texts_to_sequences([seed_text])
print("Индексированная последовательность:", sequence)

# Паддирование последовательности до фиксированной длины
max_sequence_length = 10  # Максимальная длина последовательности
padded_sequence = pad_sequences(sequence, maxlen=max_sequence_length, padding='pre')
print("Паддированная последовательность:", padded_sequence)

# Прогон через модель (предполагаем, что модель уже обучена)
# Создание простой модели для примера
model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=50, input_length=max_sequence_length),
    LSTM(100, return_sequences=True),
    LSTM(100),
    Dense(len(tokenizer.word_index) + 1, activation='softmax')
])

# Для теста прогоняем модель с паддированной последовательностью
predicted_probs = model.predict(padded_sequence)  # Вероятности для всех слов
predicted_index = np.argmax(predicted_probs, axis=-1)[0]  # Индекс предсказанного слова
print(f"Предсказанный индекс: {predicted_index}")

# Получаем слово из индекса
word = None
for w, index in tokenizer.word_index.items():
    if index == predicted_index:
        word = w
        break

print(f"Предсказанное слово: {word}")

# Прогон через модель с температурой
def predict_with_temperature(model, padded_sequence, temperature=0.7):
    predictions = model.predict(padded_sequence)[0]
    predictions = np.log(predictions + 1e-7) / temperature  # Логарифмируем и применяем температуру
    exp_preds = np.exp(predictions)
    predictions = exp_preds / np.sum(exp_preds)  # Приводим к нормализованным вероятностям
    return np.random.choice(len(predictions), p=predictions)

# Применяем функцию с температурой
predicted_index_temp = predict_with_temperature(model, padded_sequence, temperature=0.7)
print(f"Предсказанный индекс с температурой: {predicted_index_temp}")

# Проверяем все слова и их индексы в токенизаторе
print("\nВсе слова и их индексы в токенизаторе:")
for word, index in tokenizer.word_index.items():
    print(f"{word}: {index}")


Индексированная последовательность: [[2, 3, 4, 5, 6]]
Паддированная последовательность: [[0 0 0 0 0 2 3 4 5 6]]


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 752ms/step
Предсказанный индекс: 10
Предсказанное слово: hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Предсказанный индекс с температурой: 10

Все слова и их индексы в токенизаторе:
<OOV>: 1
the: 2
quick: 3
brown: 4
fox: 5
jumps: 6
over: 7
lazy: 8
dog: 9
hello: 10
world: 11
